1. Load the CSV file into a DataFrame.
2. Parse the "Radiomics" column, as it contains JSON data.
3. Remove columns with the same values across all rows.

In [123]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# create random seed for reproducibility
random_state = 42

# Load the data from DF_Radiomics_noduls_with_diagnose.csv
file_path = "DF_Radiomics_noduls_with_diagnose.csv"
data = pd.read_csv(file_path)

print(data.shape)

# Convert the 'Labels' column to an integer
data['Labels'] = data['Labels'].astype(int)

# drop all rows where the label == 0
data = data[data.Labels != 0]

# Parse the JSON in the 'Radiomics' column
data['Radiomics'] = data['Radiomics'].apply(json.loads)

# Convert the 'Radiomics' column into separate columns
radiomics_data = pd.json_normalize(data['Radiomics'])


# Drop the original 'Radiomics' column
data = data.drop('Radiomics', axis=1)
print(data.shape)

print(radiomics_data.shape)

# Reset the indices of both DataFrames
data = data.reset_index(drop=True)
radiomics_data = radiomics_data.reset_index(drop=True)

# Combine the data with the new radiomics columns
data = pd.concat([data, radiomics_data], axis=1)

print(data.shape)
# Remove columns with the same value across all rows
data = data.loc[:, (data != data.iloc[0]).any()]




(405, 4)
(309, 3)
(309, 123)
(309, 126)


In [124]:
#show  rows with NaN values
data[data.isna().any(axis=1)].head()
#count NaN values per column
print(data.isna().sum())

Patient                                                       0
Node                                                          0
Labels                                                        0
diagnostics_Image-original_Hash                               0
diagnostics_Image-original_Spacing                            0
                                                           ... 
diagnostics_Configuration_Settings.minimumROISize           309
diagnostics_Configuration_Settings.removeOutliers           309
diagnostics_Configuration_Settings.resampledPixelSpacing    309
diagnostics_Configuration_Settings.resegmentRange           309
diagnostics_Configuration_Settings.weightingNorm            309
Length: 108, dtype: int64


In [125]:
print(data.isna().sum())

Patient                                                       0
Node                                                          0
Labels                                                        0
diagnostics_Image-original_Hash                               0
diagnostics_Image-original_Spacing                            0
                                                           ... 
diagnostics_Configuration_Settings.minimumROISize           309
diagnostics_Configuration_Settings.removeOutliers           309
diagnostics_Configuration_Settings.resampledPixelSpacing    309
diagnostics_Configuration_Settings.resegmentRange           309
diagnostics_Configuration_Settings.weightingNorm            309
Length: 108, dtype: int64


In [126]:
# if a column is type string, show it
for col in data.columns:
    if data[col].dtype == 'object':
        print(col)

# type of column diagnostics_Image-original_Spacing
print(data['diagnostics_Image-original_Spacing'].dtype)

#remove all columns with type string
data = data.select_dtypes(exclude=['object'])

print(data.shape)

#explode all columns with type object
data = data.apply(pd.Series.explode).reset_index(drop=True)

print(data.shape)

Patient
Node
diagnostics_Image-original_Hash
diagnostics_Image-original_Spacing
diagnostics_Image-original_Size
diagnostics_Mask-original_Hash
diagnostics_Mask-original_Spacing
diagnostics_Mask-original_Size
diagnostics_Mask-original_BoundingBox
diagnostics_Mask-original_CenterOfMassIndex
diagnostics_Mask-original_CenterOfMass
diagnostics_Configuration_Settings.minimumROISize
diagnostics_Configuration_Settings.removeOutliers
diagnostics_Configuration_Settings.resampledPixelSpacing
diagnostics_Configuration_Settings.resegmentRange
diagnostics_Configuration_Settings.weightingNorm
object
(309, 92)
(309, 92)


In [58]:
data.shape

(384, 121)

In [59]:
#drop all rows with NaN values
data = data.dropna()

In [60]:
data.shape

(234, 121)

In [44]:
# Create a stratified split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'])

# Save the data to CSV files
train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data.csv', index=False)
test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data.csv', index=False)

ValueError: Input y contains NaN.

In [32]:
data.head(2)

,Patient,Node,Labels,diagnostics_Image-original_Hash,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,diagnostics_Configuration_Settings.minimumROISize,diagnostics_Configuration_Settings.removeOutliers,diagnostics_Configuration_Settings.resampledPixelSpacing,diagnostics_Configuration_Settings.resegmentRange,diagnostics_Configuration_Settings.weightingNorm
0,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,0506d1d0d6522eddd1640c8ea75c2fc5a9266270,...,0.053875,0.021012,0.488461,152.929922,0.019809,None,None,None,None,None
1,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,9d7da356d43e2f7ad7f374f6c193e97f6088d7c7,...,0.110650,0.024328,0.494688,165.356306,0.010062,None,None,None,None,None


In [18]:
print("Train data:", train_data.shape)
print("Test data:", test_data.shape)

Train data: (324, 108)
Test data: (81, 108)


# Fully Connected Neural Network

In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming 'data' is your pandas DataFrame
# Ensure the DataFrame only contains numeric values
data = data.apply(pd.to_numeric, errors='coerce')

# Split data into features and labels
X = data.drop('Labels', axis=1).values
y = data['Labels'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X).float()
y_tensor = torch.tensor(y).float()

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [20]:
print("Train data:", train_dataset.tensors[0].shape)

Train data: torch.Size([324, 107])


In [42]:
# Model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            #nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Hyperparameters
input_size = train_dataset.tensors[0].shape[1]  # Get the number of features from your dataset
hidden_size = 64  # You can tune this
output_size = 4   # 4 labels according to the dataset description
learning_rate = 0.001
batch_size = 32
epochs = 50  # Adjust based on your runtime requirement
clip_value = 1  # for gradient clipping

# Initialize model, loss function, and optimizer
model = FCNN(input_size, hidden_size, output_size).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 regularization

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [28]:
#check if cuda is available, print the gpu model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    # Move model to the device
    model = model.to(device)

NVIDIA GeForce RTX 3090


In [43]:
# Training loop
model.fc = nn.Linear(num_features, output_size)  # 'num_output_neurons' is the number of output neurons in your linear layer

# Training loop
model.train()
for epoch in range(epochs):
    for inputs, targets in train_loader:
        # Move inputs and targets to the device
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        # Convert targets to LongTensor
        targets = targets.long()

        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/50, Loss: nan
Epoch 2/50, Loss: nan
Epoch 3/50, Loss: nan
Epoch 4/50, Loss: nan
Epoch 5/50, Loss: nan
Epoch 6/50, Loss: nan
Epoch 7/50, Loss: nan
Epoch 8/50, Loss: nan
Epoch 9/50, Loss: nan
Epoch 10/50, Loss: nan
Epoch 11/50, Loss: nan
Epoch 12/50, Loss: nan
Epoch 13/50, Loss: nan
Epoch 14/50, Loss: nan
Epoch 15/50, Loss: nan
Epoch 16/50, Loss: nan
Epoch 17/50, Loss: nan
Epoch 18/50, Loss: nan
Epoch 19/50, Loss: nan
Epoch 20/50, Loss: nan
Epoch 21/50, Loss: nan
Epoch 22/50, Loss: nan
Epoch 23/50, Loss: nan
Epoch 24/50, Loss: nan
Epoch 25/50, Loss: nan
Epoch 26/50, Loss: nan
Epoch 27/50, Loss: nan
Epoch 28/50, Loss: nan
Epoch 29/50, Loss: nan
Epoch 30/50, Loss: nan
Epoch 31/50, Loss: nan
Epoch 32/50, Loss: nan
Epoch 33/50, Loss: nan
Epoch 34/50, Loss: nan
Epoch 35/50, Loss: nan
Epoch 36/50, Loss: nan
Epoch 37/50, Loss: nan
Epoch 38/50, Loss: nan
Epoch 39/50, Loss: nan
Epoch 40/50, Loss: nan
Epoch 41/50, Loss: nan
Epoch 42/50, Loss: nan
Epoch 43/50, Loss: nan
Epoch 44/50, Loss: n

In [44]:
# Evaluate the model
# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient calculations
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability

            total_predictions += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

    accuracy = correct_predictions / total_predictions
    return accuracy

# Use the function
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
accuracy = evaluate(model, test_loader, device)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 23.456790123456788%
